# Code for scraping data from dynamic pages into csv files and creating a tarfile out of it. 

## Imports and scrapfun() to scrape company names

In [51]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
from selenium.webdriver.firefox.options import Options
import os
import requests
import csv
from bs4 import BeautifulSoup


def scrapfunc(page,filename):
    f = csv.writer(open(filename, 'w'))
    f.writerow(['Symbol', 'Security'])
    soup = BeautifulSoup(page.text, 'html.parser')
    #print(soup.prettify())
    My_table = soup.find('table',{'class':'wikitable sortable'})
    #print(My_table)
    tr=My_table.findAll('tr')
    for row in tr:
        #print(row)
        print("\n")
        td=row.findAll('td')
        #print(td,type(td))
        j=0
        sym=0
        sec=0
        for i in td:
            if j%2==0:
                sym=i.contents[0].contents[0]
            else:
                sec=i.contents[0].contents[0]
            j+=1
            if j==2:
                break
        f.writerow([sym,sec])
        print("\n")

## Calling scrapefun()

In [52]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

scrapfunc(page,'sym_sec_table.csv')


## Creating a dataframe of symbol and security from scraped data

In [53]:
import pandas as pd
datafr=pd.read_csv('sym_sec_table.csv')
datafr

,Symbol,Security
0,0,0
1,MMM,3M Company
2,ABT,Abbott Laboratories
3,ABBV,AbbVie Inc.
4,ABMD,ABIOMED Inc
5,ACN,Accenture plc
6,ATVI,Activision Blizzard
7,ADBE,Adobe Systems Inc
8,AMD,Advanced Micro Devices Inc
9,AAP,Advance Auto Parts


## Defining a function to scrape data from dynamic pages of the company using Selenium

In [54]:
from selenium.webdriver.firefox.options import Options
from contextlib import contextmanager
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
def scrap_com_data(sym,filename):
    
    
    
    url='https://stockrow.com/'+str(sym)
    #Options().headless=True
    driver = webdriver.Firefox()
    #
    #time.sleep(20)
    
    driver.get(url)
    #driver.implicitly_wait(40)
    
    timeout = 5
    try:
        element_present1 = EC.presence_of_element_located((By.CLASS_NAME, 'keystats-overview'))
        element_present2 = EC.presence_of_element_located((By.CLASS_NAME, 'keystats-data'))
        WebDriverWait(driver, timeout).until(element_present1)
        WebDriverWait(driver, timeout).until(element_present2)
    except TimeoutException:
        print ("Timed out waiting for page to load")
    
    soup=BeautifulSoup(driver.page_source, 'lxml')
    driver.quit()
    
    
    f = csv.writer(open(filename, 'w'))
#print(soup.prettify())
#if 1==1:
    My_table = soup.find('table',{'class':'keystats-overview'})
    #print(My_table)
    label=My_table.findAll('div',{'class':'cell-label'})
    value=My_table.findAll('div',{'class':'cell-value'})
    for i,j in zip(label,value):
        f.writerow([i.getText(),j.getText()])
        
        
        
    My_table = soup.find('table',{'class':'keystats-data'})
    #print(My_table)
    #head=My_table.findAll('th')
    head_row=My_table.findAll('th')
    list=[]
    for h in head_row:
        list.append(h.getText())
    f.writerow(list)
    #print("\n")
    
    tabrows=My_table.findAll('tr')
    for row in tabrows:
        row_data=row.findAll('td')
        list=[]
        for data in row_data:
            list.append(data.getText())
        f.writerow(list)
    #for i,j in zip(label,value):
        #print("\n")

## Calling scrap_com_data() to scrape dynamic pages

In [55]:
j=0
l=[]
count=3
for sym in datafr['Symbol']:
    #try:
    
    if sym!=str('0'):
        l.append(sym)
        scrap_com_data(sym,str(sym)+str('.csv'))
    #except:
     #   print("skipped")
    j+=1
    if j>count:
        break

## creating a tarfile out of the scraped data

In [56]:
import tarfile
tar = tarfile.open("sample.tar.gz", "w:gz")
for name in l:
    tar.add(str(name)+str('.csv'))
tar.close()